In [1]:
!pip install roboflow tqdm -q

import os
import cv2
import shutil
import yaml
from roboflow import Roboflow
from tqdm import tqdm
from IPython.display import FileLink

rf = Roboflow(api_key="EBfUGkJAAgSJn1GrpmXD")
project = rf.workspace("trn-xun-c").project("human-face-emotions-dxxni")
version = project.version(1)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 40.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 105.9 MB/s eta 0:00:0000:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.47.0 which is incompatible.
google-colab 1.0.0 requires jupyter-server==2.14.0, but you have jupyter-server 2.12.5 which is incompatible.
google-colab 1.0.0 r


Extracting Dataset Version Zip to Human-face-emotions-1 in yolov8:: 100%|██████████| 28728/28728 [00:02<00:00, 9744.25it/s] 


In [2]:
CLASSES = ['anger', 'content', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

output_dir = '/kaggle/working/cropped_dataset'

for split in ['train', 'valid', 'test']:
    for cls in CLASSES:
        os.makedirs(os.path.join(output_dir, split, cls), exist_ok=True)

print("✅ Đã tạo cấu trúc thư mục xong!")

✅ Đã tạo cấu trúc thư mục xong!


In [3]:
def process_split(split_name):
    # Đường dẫn ảnh và nhãn gốc
    images_path = f"{dataset.location}/{split_name}/images"
    labels_path = f"{dataset.location}/{split_name}/labels"
    
    # Lấy danh sách file
    image_files = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    print(f"✂️ Đang xử lý tập {split_name} ({len(image_files)} ảnh)...")
    
    count = 0
    for img_name in tqdm(image_files):
        # Đọc ảnh
        img_path = os.path.join(images_path, img_name)
        img = cv2.imread(img_path)
        h_img, w_img, _ = img.shape
        
        # Đọc nhãn tương ứng (.txt)
        label_name = img_name.rsplit('.', 1)[0] + '.txt'
        label_path = os.path.join(labels_path, label_name)
        
        if not os.path.exists(label_path):
            continue
            
        with open(label_path, 'r') as f:
            lines = f.readlines()
            
        # Xử lý từng khuôn mặt trong ảnh
        for i, line in enumerate(lines):
            # Format YOLO: class x_center y_center width height (tất cả là tỉ lệ 0-1)
            data = line.strip().split()
            cls_idx = int(data[0])
            x_center, y_center, w, h = map(float, data[1:])
            
            # Chuyển đổi sang tọa độ pixel
            x1 = int((x_center - w/2) * w_img)
            y1 = int((y_center - h/2) * h_img)
            x2 = int((x_center + w/2) * w_img)
            y2 = int((y_center + h/2) * h_img)
            
            # Giới hạn tọa độ trong khung hình (tránh lỗi cắt ra ngoài ảnh)
            x1 = max(0, x1); y1 = max(0, y1)
            x2 = min(w_img, x2); y2 = min(h_img, y2)
            
            # Cắt ảnh khuôn mặt
            face_crop = img[y1:y2, x1:x2]
            
            # Bỏ qua ảnh lỗi hoặc quá bé (<10 pixel)
            if face_crop.size == 0 or face_crop.shape[0] < 10 or face_crop.shape[1] < 10:
                continue
            
            # Lưu ảnh mặt vào folder tương ứng
            cls_name = CLASSES[cls_idx]
            save_name = f"{img_name.rsplit('.', 1)[0]}_face{i}.jpg"
            save_path = os.path.join(output_dir, split_name, cls_name, save_name)
            
            cv2.imwrite(save_path, face_crop)
            count += 1
            
    print(f"   -> Đã cắt được {count} khuôn mặt từ tập {split_name}.")

In [4]:
process_split('train')
process_split('valid')
process_split('test')

✂️ Đang xử lý tập train (11544 ảnh)...


100%|██████████| 11544/11544 [00:33<00:00, 345.54it/s]


   -> Đã cắt được 11703 khuôn mặt từ tập train.
✂️ Đang xử lý tập valid (1873 ảnh)...


100%|██████████| 1873/1873 [00:05<00:00, 346.15it/s]


   -> Đã cắt được 1902 khuôn mặt từ tập valid.
✂️ Đang xử lý tập test (941 ảnh)...


100%|██████████| 941/941 [00:02<00:00, 348.20it/s]

   -> Đã cắt được 955 khuôn mặt từ tập test.


In [5]:
print("📦 Đang nén dataset khuôn mặt...")
shutil.make_archive('Dataset_Mat_Cat', 'zip', output_dir)
print("🎉 Xong! Bạn có thể tải về để xem thử.")
FileLink('Dataset_Mat_Cat.zip')

📦 Đang nén dataset khuôn mặt...
🎉 Xong! Bạn có thể tải về để xem thử.


/kaggle/working/Dataset_Mat_Cat.zip